In [1]:
import sys
import pandas as pd
sys.path.insert(0, '..')
from src.eventstream import Eventstream, EventstreamSchema, RawDataSchema
from src.tooling.step_matrix import StepMatrix
import pytest
import os

/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


ПРИ ПРОГОНЕ ТУТОРИАЛА НЕ ОТОБРАЖАЛАСЬ СТРОЧКА ENDED. И в правильных ответах на тесты её тоже нет.
1) Шаги
    1. Количество шагов у каждого пользователя в датасете [2, 100]
    2. Максимальное количество шагов в матрице [0, 2, 16, 50, 100, 200]
    3. Дробное число шагов? Отрицательное? ("ValueError: max() arg is an empty sequence"). Ноль это тоже отрицательный тест!
2) Пороги
    1. Отрицательные, целые, ноль, один и т.д. [-50, -0.02, 0, 0.1, 0.05, 1, 67, 1000]
    2. При прогоне туториала в 
3)

In [2]:
# базовый датасет в формате датафрейма
raw_data = pd.read_csv("../src/datasets/data/simple-onlineshop.csv")

In [3]:
#пользователи отсортированные по количеству cобытий
raw_data.groupby(by='user_id', as_index=False).count().sort_values(by=['event'])

,user_id,event,timestamp
3750,999941967,2,2
447,132638296,2,2
1927,517179956,2,2
1941,519785837,2,2
1944,521074614,2,2
...,...,...,...
2881,760510988,83,83
265,78179456,83,83
3377,898482504,92,92
3621,962952209,94,94


In [4]:
min_table = raw_data.loc[raw_data['user_id'] == 999941967] #пользователь с двумя событиями

In [5]:
minmax_table = min_table.append(raw_data.loc[raw_data['user_id'] == 530349536], ignore_index = True) # минимум и максимум вместе 
minmax_table

,user_id,event,timestamp
0,999941967,catalog,2020-04-17 12:18:49.391199
1,999941967,lost,2020-04-17 12:18:50.391199
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
...,...,...,...
97,530349536,main,2020-03-15 03:08:17.586834
98,530349536,catalog,2020-03-15 03:08:18.914158
99,530349536,cart,2020-03-15 03:08:23.925717
100,530349536,delivery_choice,2020-03-15 03:08:27.382559


In [6]:
two_users_table = minmax_table[:8] # 2 пользователя, максимум 6 шагов

In [7]:
# create data schema
raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

# create source eventstream
source = Eventstream(
    raw_data=two_users_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

## Max steps

При проведении тестов логичнее пользоваться экземпляром класса StepMatrix, т.к. если записывать его как метод eventstream, он автоматически вызывает функцию plot(), которая мешает интерпретации результатов

In [8]:
#max step == 6
step_matrix = StepMatrix(eventstream=source, 
                          max_steps=6)
result = step_matrix._get_plot_data()[0]
display(result)
result.columns

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


Int64Index([1, 2, 3, 4, 5, 6], dtype='int64')

In [9]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.5, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.5, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.5, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.5],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0]
                
            ], index=['catalog', 'cart', 'delivery_choice', 'delivery_courier',
       'payment_choice', 'payment_card', 'lost'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

,1,2,3,4,5,6
catalog,1.0,0.0,0.0,0.0,0.0,0.0
cart,0.0,0.5,0.0,0.0,0.0,0.0
delivery_choice,0.0,0.0,0.5,0.0,0.0,0.0
delivery_courier,0.0,0.0,0.0,0.5,0.0,0.0
payment_choice,0.0,0.0,0.0,0.0,0.5,0.0
payment_card,0.0,0.0,0.0,0.0,0.0,0.5
lost,0.0,0.5,0.0,0.0,0.0,0.0


In [10]:
#assert
def compare_results(res, corr_res):
    return res.compare(corr_res).shape == (0, 0)
compare_results(result, correct_result)

True

In [11]:
#max step == 1
correct_result = pd.DataFrame(
            [
                [1.0]
            ], index=['catalog'], columns=[1],
        )
correct_result

,1
catalog,1.0


In [12]:
step_matrix = StepMatrix(eventstream=source, 
                          max_steps=1)
result = step_matrix._get_plot_data()[0]
result

,1
catalog,1.0


In [13]:
#assert
result.compare(correct_result).shape == (0, 0)

True

In [14]:
# ещё одна таблица из старых тестов
# если в этих файлах хранятся правильные ответы, то можно использовать и её: 01_basic.csv, 02_one_step.csv, 03_100_steps.csv

source_event123 = pd.DataFrame(
            [
                [1, "event1", "2022-01-01 00:01:00"],
                [1, "event2", "2022-01-01 00:01:02"],
                [1, "event1", "2022-01-01 00:02:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event3", "2022-01-01 00:03:30"],
                [1, "event1", "2022-01-01 00:04:00"],
                [1, "event3", "2022-01-01 00:04:30"],
                [1, "event1", "2022-01-01 00:05:00"],
                [2, "event1", "2022-01-02 00:00:00"],
                [2, "event2", "2022-01-02 00:00:05"],
                [2, "event2", "2022-01-02 00:01:05"],
                [3, "event1", "2022-01-02 00:01:10"],
                [3, "event1", "2022-01-02 00:02:05"],
                [3, "event4", "2022-01-02 00:03:05"],
                [4, "event1", "2022-01-02 00:01:10"],
                [4, "event1", "2022-01-02 00:02:05"],
                [4, "event1", "2022-01-02 00:03:05"],
            ],
            columns=["user_id", "event", "timestamp"],
        )
source_event123

,user_id,event,timestamp
0,1,event1,2022-01-01 00:01:00
1,1,event2,2022-01-01 00:01:02
2,1,event1,2022-01-01 00:02:00
3,1,event1,2022-01-01 00:03:00
4,1,event1,2022-01-01 00:03:00
5,1,event3,2022-01-01 00:03:30
6,1,event1,2022-01-01 00:04:00
7,1,event3,2022-01-01 00:04:30
8,1,event1,2022-01-01 00:05:00
9,2,event1,2022-01-02 00:00:00


In [15]:
FLOAT_PRECISION = 6 # здесь дроби будут с большим числом знаков после запятой

In [16]:
def read_test_data(filename):
    # да, файлы с правильными ответами лежат в другой папке
    filepath = os.path.join("../tests/tooling/step_matrix_test_data", filename)
    df = pd.read_csv(filepath, index_col=0).round(FLOAT_PRECISION)
    df.columns = df.columns.astype(int)
    return df

In [17]:
# всё открывается
# read_test_data('01_basic.csv')

In [18]:
@pytest.fixture
def stream():
    def remove_start(df, schema):
        return df["event_name"] != "start"

    test_stream = datasets.load_simple_shop()
    graph = PGraph(source_stream=test_stream)
    node1 = EventsNode(StartEndEvents(params=StartEndEventsParams(**{})))
    node2 = EventsNode(FilterEvents(params=FilterEventsParams(filter=remove_start)))

    graph.add_node(node=node1, parents=[graph.root])
    graph.add_node(node=node2, parents=[node1])

    stream = graph.combine(node=node2)
    return stream

In [19]:
def run_test(stream, filename, **kwargs):
    sm = StepMatrix(eventstream=stream, **kwargs)
    result = sm._get_plot_data()[0]
    result = result.round(FLOAT_PRECISION)
    result_correct = read_test_data(filename)
    test_is_correct = result.compare(result_correct).shape == (0, 0)
    return test_is_correct

In [20]:
class TestStepMatrix:
    def test_step_matrix_simple(self):
        source_df = pd.DataFrame(
            [
                [1, "event1", "2022-01-01 00:01:00"],
                [1, "event2", "2022-01-01 00:01:02"],
                [1, "event1", "2022-01-01 00:02:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event3", "2022-01-01 00:03:30"],
                [1, "event1", "2022-01-01 00:04:00"],
                [1, "event3", "2022-01-01 00:04:30"],
                [1, "event1", "2022-01-01 00:05:00"],
                [2, "event1", "2022-01-02 00:00:00"],
                [2, "event2", "2022-01-02 00:00:05"],
                [2, "event2", "2022-01-02 00:01:05"],
                [3, "event1", "2022-01-02 00:01:10"],
                [3, "event1", "2022-01-02 00:02:05"],
                [3, "event4", "2022-01-02 00:03:05"],
                [4, "event1", "2022-01-02 00:01:10"],
                [4, "event1", "2022-01-02 00:02:05"],
                [4, "event1", "2022-01-02 00:03:05"],
            ],
            columns=["user_id", "event", "timestamp"],
        )

        correct_result = pd.DataFrame(
            [[1.0, 0.5, 0.5, 0.25, 0.25], [0.0, 0.5, 0.25, 0.0, 0.0], [0.0, 0.0, 0.25, 0.0, 0.0]],
            index=["event1", "event2", "event4"],
            columns=[1, 2, 3, 4, 5],
        )

        source_stream = Eventstream(
            raw_data=source_df,
            raw_data_schema=RawDataSchema(event_name="event", event_timestamp="timestamp", user_id="user_id"),
            schema=EventstreamSchema(),
        )

        sm = StepMatrix(eventstream=source_stream, max_steps=5)
        result= sm._get_plot_data()[0]
        assert result.compare(correct_result).shape == (0, 0)

    def test_step_matrix__basic(self, stream):
        assert run_test(stream, "01_basic.csv")

    def test_step_matrix__one_step(self, stream):
        assert run_test(stream, "02_one_step.csv", max_steps=1)

    def test_step_matrix__100_steps(self, stream):
        assert run_test(stream, "03_100_steps.csv", max_steps=100, precision=3)

In [21]:
# не запускается(

## Tresholds

In [22]:
source_event123.head()

,user_id,event,timestamp
0,1,event1,2022-01-01 00:01:00
1,1,event2,2022-01-01 00:01:02
2,1,event1,2022-01-01 00:02:00
3,1,event1,2022-01-01 00:03:00
4,1,event1,2022-01-01 00:03:00


In [23]:
#подготовить данные для теста
data_for_tresh = source_event123
# create data schema
raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

# create source eventstream
source_tresh = Eventstream(
    raw_data=data_for_tresh,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [24]:
#let's look at the matrix without using threshold
StepMatrix(eventstream=source_tresh, 
                          max_steps=6)._get_plot_data()[0]

,1,2,3,4,5,6
event1,1.0,0.5,0.50,0.25,0.25,0.00
event2,0.0,0.5,0.25,0.00,0.00,0.00
event4,0.0,0.0,0.25,0.00,0.00,0.00
event3,0.0,0.0,0.00,0.00,0.00,0.25


In [25]:
# after thresholding the row  THRESHOLDED_X must appear. If we'll choose thresh == 0.3, number of thresholded events will be 4
# event3 appers later then event4, it's important for future correct results dataframe

In [26]:
# create an insistance of StepMatrix class and save output data
# thresh=0.3
step_matrix = StepMatrix(eventstream=source_tresh, 
                          max_steps=6, thresh=0.3)
result = step_matrix._get_plot_data()[0]
result
#падает всё с той же ошибкой

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.5, 0.5, 0.0, 0.0, 0.0],
                [0.0, 0.5, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.5, 0.25, 0.25, 0.25]
                
            ], index=['event1', 'event2', 'event4', 'event3', 'THRESHOLDED_4'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result

In [ ]:
#assert
result.compare(correct_result).shape == (0, 0)

In [ ]:
# create an insistance of StepMatrix class and save output data
# thresh=1.0
step_matrix = StepMatrix(eventstream=source_tresh, 
                          max_steps=6, thresh=1.0)
result = step_matrix._get_plot_data()[0]

In [ ]:
correct_result = pd.DataFrame(
            [
                [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 1.0, 0.25, 0.25, 0.25]
                
            ], index=['event1', 'event2', 'event4', 'event3', 'THRESHOLDED_4'], columns=[1, 2 , 3, 4, 5, 6],
        )
correct_result